In [1]:
### PROCEDURES ###
#   Configerations
#        |
#        |
# Galaxy distribution
#        |
#        |
#    CosmoSIS
#        |
#        |
#    get gal distr (HOME)
#    get chi       (COSMOSIS)
#    get h         (COSMOSIS)
#    get P_k_h non_lin    (COSMOSIS)
#    get C_gg, C_GG, C_Gg (COSMOSIS)
#    get C_IG, C_II, C_Ig (COSMOSIS) #pending
#        |
#        |
# Compute lensing functions
#        |

In [1]:
print('Started. Loading libraries ... ')
import os,sys,inspect
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
import pandas as pd
import fitsio as fio
import pickle
from copy import copy, deepcopy

from interpolation_fncs import *
from io_fncs import *
from gal_distr import *
from basic_cosmo_lensing_fncs import *
# from limber_integral import *



########################################
#io
########################################
outputfolder       = 'chi_based_SC'
outputparentfolder = '/hpc/group/cosmology/alanzhou/IA-Project/IA-Self-Calib-Data/'
outputpath         = os.path.join(outputparentfolder, outputfolder)
create_dir(outputpath)
print(' - output path set to %s'%(outputpath))



########################################
#constants
########################################
light_speed = 299792.458 #km/s
H0 = 67.26 #H0 (km/s/Mpc)
d_horiz = light_speed/H0 #c/H0, c in km/s
Omega0 = 0.3141 #present day matter density in unit of critical density
OmegaLambda = 1-0.3141
Omegak = 1-Omega0-OmegaLambda
deltaz = 0.2 #redshift bin width
#in integration 1 redshift is broken into z_res intervals
#for z_max = 4, zres = 400 ~20sec/run
z_bin_num = 20
z_max = z_bin_num*deltaz+2 #+2 redshift reserved for photometric error
z_res = 100 
gal_distr_model = 'z2exp_gausPDF'

save_config_file(os.path.join(outputpath,'values.txt'),
                 ['Identifier',outputfolder,
                  'H0, [km/s/Mpc]',H0,
                  'horizon distance = c/H0, [Mpc]',d_horiz,
                  'speed of light [km/s]',light_speed,
                  'Omega0',Omega0,
                  'OmegaLambda',OmegaLambda,
                  'Omegak',Omegak,
                  'deltaz',deltaz,
                  'number of galaxy redshift bins',z_bin_num,
                  'max z used in calculations',z_max,
                  'z resolution [intervals/ unit z]',z_res,
                  'z resolution spacing [z]',1/z_res,
                  'galaxy distribution model',gal_distr_model,
                  'output path', outputpath])

print(' - Setup complete. %i photometric redshift bins; computational z resolution is %1.5f.' %(z_bin_num,1/z_res))



########################################################
#Preliminary calculation: gal distribution + CosmoSIS. #
########################################################
#get the galaxy distribution
print('Getting the galaxy distribution, assuming ' + gal_distr_model)
get_gal_distr(z_bin_num,deltaz,z_res,z_max,gal_distr_model,outputpath)

#run CosmoSIS
#run the pow_specs.ini with CosmoSIS from terminal
#the cosmoSIS has to be set up with the same cosmology
#the max z, max ell in CAMB must be at least span the Z_MID's range
print('Assuming CosmoSIS has been run. Starting data analysis...')



########################################################
#Loading data from prev calculations and CosmoSIS
########################################################
print('Loading data...')



print(' - Loading the galaxy distribution')
gal_distr_folder = os.path.join(outputpath,'n_z')
n_true_z_file = open(os.path.join(gal_distr_folder,'n_true_z.pkl'), 'rb')
n_true_z_data = pickle.load(n_true_z_file)
n_true_z_file.close()
n_photo_z_file = open(os.path.join(gal_distr_folder,'n_photo_z.pkl'), 'rb')
n_photo_z_data = pickle.load(n_photo_z_file)
n_photo_z_file.close()

Z_MID = n_true_z_data['Z_MID']
if not np.array_equal(Z_MID,n_photo_z_data['Z_MID']):raise Exception("true z Z_MID incompatible with photo z Z_MID")

    
    
print(' - Loading distances from CosmoSIS')
# z_CosmoSIS         [redshift]
# chi_CosmoSIS       [Mpc]
# little_h_CosmoSIS  [100 km/s/Mpc] 
distance_folder_CosmoSIS = os.path.join(outputpath,'cosmoSIS_output/distances')
with open(os.path.join(distance_folder_CosmoSIS,'z.txt'),'rb') as f:
    z_CosmoSIS = np.loadtxt(f)
with open(os.path.join(distance_folder_CosmoSIS,'d_m.txt'),'rb') as f:
    chi_CosmoSIS = np.loadtxt(f)
with open(os.path.join(distance_folder_CosmoSIS,'h.txt'),'rb') as f:
    little_h_CosmoSIS = np.loadtxt(f)
little_h_CosmoSIS = little_h_CosmoSIS*light_speed/100



print(' - Loading non-linear P(k,z) from CosmoSIS')
#get the power spectrum P(k,z) from CosmoSIS. (need non-linear spectrum)
#CosmoSIS unit: k is in (Mpc/h), P is in (h/Mpc)^3
#interpolating the P(k,z) to fit Z_MID
#matter_pow_p_k[z_index,kindex]
matter_power_folder = os.path.join(outputpath,'cosmoSIS_output/matter_power_nl')
with open(os.path.join(matter_power_folder, 'p_k.txt'),'rb') as f:
    matter_pow_p_k = np.loadtxt(f,delimiter = ' ') 
with open(os.path.join(matter_power_folder, 'k_h.txt'),'rb') as f:
    matter_pow_k_h = np.loadtxt(f,delimiter = ' ')
with open(os.path.join(matter_power_folder, 'z.txt'),'rb') as f:
    matter_pow_z = np.loadtxt(f,delimiter = ' ')



#load the angular GG,Gg,gg speactra from CosmoSIS
#the ell coordinates from the CosmoSIS outputs must be identical across spectra
#load the C_GG in angular space from CosmoSIS (shear-shear option)
print(' - Loading C_GG(ell) from CosmoSIS')
shear_cl_folder = os.path.join(outputpath,'cosmoSIS_output/shear_cl')
with open(os.path.join(shear_cl_folder,'ell.txt'),'rb') as f:
    ell_list = np.loadtxt(f)
C_GG_ii = np.empty((z_bin_num,len(ell_list)))
plt.subplots(figsize=(6,6))
for i in range(z_bin_num):
    with open(os.path.join(shear_cl_folder,'bin_%i_%i.txt'%(i+1,i+1)),'rb') as f:
        C_GG_ii[i] = np.loadtxt(f)
    plt.plot(ell_list,C_GG_ii[i],label='C_GG_ii, i=%i'%(i+1))
plt.xlabel('ell')
plt.ylabel('C_GG_ii')
plt.xscale('log')
plt.yscale('log')
plt.legend(bbox_to_anchor=(1, 0),loc='lower left') #bbox(x,y)
plt.tight_layout()
plt.savefig(os.path.join(outputpath,'combined_C_GG_ii.png'))
plt.close()

#load the C_Gg in angular space from CosmoSIS (position-shear option)
print(' - Loading C_Gg(ell) from CosmoSIS')
galaxy_shear_cl_folder = os.path.join(outputpath,'cosmoSIS_output/galaxy_shear_cl')
with open(os.path.join(galaxy_shear_cl_folder,'ell.txt'),'rb') as f:
    ell_list_test = np.loadtxt(f)
if not np.array_equal(ell_list, ell_list_test): raise Exception('ell coordinates differ for different CosmoSIS angular spectra')
C_Gg_ii = np.empty((z_bin_num,len(ell_list)))
plt.subplots(figsize=(6,6))
for i in range(z_bin_num):
    with open(os.path.join(galaxy_shear_cl_folder,'bin_%i_%i.txt'%(i+1,i+1)),'rb') as f:
        C_Gg_ii[i] = np.loadtxt(f)
    plt.plot(ell_list,C_Gg_ii[i],label='C_Gg_ii, i=%i'%(i+1))
plt.xlabel('ell')
plt.ylabel('C_Gg_ii')
plt.xscale('log')
plt.yscale('log')
plt.legend(bbox_to_anchor=(1, 0),loc='lower left') #bbox(x,y)
plt.tight_layout()
plt.savefig(os.path.join(outputpath,'combined_C_Gg_ii.png'))
plt.close()

#load the C_gg in angular space from CosmoSIS (position-position option)
print(' - Loading C_gg(ell) from CosmoSIS')
galaxy_cl_folder = os.path.join(outputpath,'cosmoSIS_output/galaxy_cl')
with open(os.path.join(galaxy_cl_folder,'ell.txt'),'rb') as f:
    ell_list_test = np.loadtxt(f)
if not np.array_equal(ell_list, ell_list_test): raise Exception('ell coordinates differ for different CosmoSIS angular spectra')
C_gg_ii = np.empty((z_bin_num,len(ell_list)))
plt.subplots(figsize=(6,6))
for i in range(z_bin_num):
    with open(os.path.join(galaxy_cl_folder,'bin_%i_%i.txt'%(i+1,i+1)),'rb') as f:
        C_gg_ii[i] = np.loadtxt(f)
    plt.plot(ell_list,C_gg_ii[i],label='C_gg_ii, i=%i'%(i+1))
plt.xlabel('ell')
plt.ylabel('C_gg_ii')
plt.xscale('log')
plt.yscale('log')
plt.legend(bbox_to_anchor=(1, 0),loc='lower left') #bbox(x,y)
plt.tight_layout()
plt.savefig(os.path.join(outputpath,'combined_C_gg_ii.png'))
plt.close()




Started. Loading libraries ... 
 - output path set to /hpc/group/cosmology/alanzhou/IA-Project/IA-Self-Calib-Data/chi_based_SC
 - Setup complete. 20 photometric redshift bins; computational z resolution is 0.01000.
Getting the galaxy distribution, assuming z2exp_gausPDF
Assuming CosmoSIS has been run. Starting data analysis...
Loading data...
 - Loading the galaxy distribution
 - Loading distances from CosmoSIS
 - Loading non-linear P(k,z) from CosmoSIS
 - Loading C_GG(ell) from CosmoSIS
 - Loading C_Gg(ell) from CosmoSIS
 - Loading C_gg(ell) from CosmoSIS


In [19]:
########################################
#Preliminary Calculations
########################################
print('Starting preliminary calculations')
create_dir(os.path.join(outputpath,'Preliminary_calculations'))



print('   - Converting distances data into functions')
z_chi_func = interpolate.interp1d(chi_CosmoSIS,z_CosmoSIS,kind='linear') 
little_h_chi_func = interpolate.interp1d(chi_CosmoSIS,little_h_CosmoSIS,kind='linear') 
chi_z_func = interpolate.interp1d(z_CosmoSIS,chi_CosmoSIS,kind='linear') 

#Constructing the chi_axis vector
#All subsequent calculations will be done on chi_axis as the distance measure
chi_Z_MID = chi_z_func(Z_MID) #the corresponding chi values for Z_MID
#re-regularize the chi axis to be of equal interval in the same definition domain with same number of histogram bins
#this will be used in the galaxy distribution file
CHI_MID = (np.arange(len(Z_MID))+0.5)*chi_z_func(max(Z_MID))/len(Z_MID)
chi_interval = CHI_MID[1]-CHI_MID[0] #uniform intervals by construnction, width of each histogram bin in gal distr.
print('   - chi interval used is %1.3f'%(chi_interval))

fig = plt.figure(figsize=(6,6))
plt.plot(z_CosmoSIS,chi_CosmoSIS,linestyle='dashed',color='black',label='CosmoSIS original output')
plt.plot(Z_MID,chi_Z_MID,color='red',label='interpolated at Z_MID')
plt.xlabel('redshift z')
plt.title('chi [Mpc]')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(outputpath,'Preliminary_calculations/chi_z.png'))
plt.close()

fig = plt.figure(figsize=(6,6))
ax1 = fig.add_subplot(211)
ax1.plot(z_CosmoSIS,little_h_CosmoSIS,color='black',label='CosmoSIS original output')
plt.legend()
ax1.set_xlabel('z')
ax2 = fig.add_subplot(212)
ax2.plot(chi_z_func(z_CosmoSIS),little_h_CosmoSIS,color='black',linestyle='dashed',label='h_cosmosis vs. chi(z_cosmosis)')
ax2.plot(CHI_MID,little_h_chi_func(CHI_MID),color='red',marker='o',markersize=4,label='h vs. chi at equal interval')
ax2.set_xlabel('chi [Mpc]')
plt.suptitle('h [100 km/s/Mpc]')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(outputpath,'Preliminary_calculations/little_h.png'))
plt.close()



print('   - Converting galaxy distribution in z-space to chi-space')
#use chi_axis to regularize the galaxy distribution
#change the table header for the chi datas to CHI_LOW, CHI_MID, CHI_HIGH...

# gal_arr_names = list(n_true_z_data.dtype.names)[:] #create a deep copy, so that subsequent operations do not modify the original names in z files
# gal_arr_names[0:3] = ['CHI_LOW','CHI_MID','CHI_HIGH']

#there is some deep copy bug in numpy? 
#np copy still links datatype names, so need to convert to pd and then connvert back
n_true_chi_data_pd = pd.DataFrame(n_true_z_data)
n_true_chi_data_pd = n_true_chi_data_pd.rename(columns={"Z_LOW": "CHI_LOW", "Z_MID": "CHI_MID", "Z_HIGH": "CHI_HIGH"},errors='raise')
n_true_chi_data_pd['CHI_MID']  = CHI_MID
n_true_chi_data_pd['CHI_LOW']  = CHI_MID-chi_interval/2
n_true_chi_data_pd['CHI_HIGH'] = CHI_MID+chi_interval/2
n_true_chi_data = n_true_chi_data_pd.to_records(index = False) #convert back to numpy array without the row indices

for bin_index in range(z_bin_num):
    gal_distr_chi_func_bin_i = interpolate.interp1d(chi_Z_MID,n_true_chi_data['BIN%i'%(bin_index+1)],kind='linear',fill_value='extrapolate') #interp1d(x,f(x))
    n_true_chi_data['BIN%i'%(bin_index+1)] = gal_distr_chi_func_bin_i(CHI_MID)


n_photo_chi_data_pd = pd.DataFrame(n_photo_z_data)
n_photo_chi_data_pd = n_photo_chi_data_pd.rename(columns={"Z_LOW": "CHI_LOW", "Z_MID": "CHI_MID", "Z_HIGH": "CHI_HIGH"},errors='raise')
n_photo_chi_data_pd['CHI_MID']  = CHI_MID
n_photo_chi_data_pd['CHI_LOW']  = CHI_MID-chi_interval/2
n_photo_chi_data_pd['CHI_HIGH'] = CHI_MID+chi_interval/2
n_photo_chi_data = n_photo_chi_data_pd.to_records(index = False) #convert back to numpy array without the row indices

for bin_index in range(z_bin_num):
    gal_distr_chi_func_bin_i = interpolate.interp1d(chi_Z_MID,n_photo_chi_data['BIN%i'%(bin_index+1)],kind='linear',fill_value='extrapolate') #interp1d(x,f(x))
    n_photo_chi_data['BIN%i'%(bin_index+1)] = gal_distr_chi_func_bin_i(CHI_MID)

plt.subplots(figsize=(8,6))
for i in range(z_bin_num):
    plt.plot(CHI_MID, n_photo_chi_data['BIN%i'%(i+1)],label = 'PHOTO-CHI BIN%i'%(i+1))
    plt.plot(CHI_MID, n_true_chi_data['BIN%i'%(i+1)],linestyle='dashed',label = 'TRUE-CHI  BIN%i'%(i+1))
plt.xlabel('Chi [Mpc]')
plt.ylabel('galaxy redshift distribution')
plt.title('n(chi), normalized for each bin seperately')
#plt.legend(bbox_to_anchor=(1, 1.05))
plt.tight_layout()
plt.savefig(os.path.join(outputpath,'n_z/combined_n_chi.png'))
plt.close()




    ########################################
    #Computing lensing kernels
    ########################################
create_dir(os.path.join(outputpath,'lensing_kernels'))

#W_L_list[chi_L,chi_G], where chi in unit of [Mpc]
W_L_list = np.empty((len(CHI_MID),len(CHI_MID)))
for chi_L_index,chi_L in enumerate(CHI_MID):
    #flat geometry (Troxel Ishak 3pt SC, eqn4)
    W_L_list[chi_L_index] = (1/d_horiz**2)*1.5*Omega0*(1+z_chi_func(chi_L))*chi_L*(1-chi_L/CHI_MID)
W_L_list[W_L_list<0]=0 #set negative entries to 0 (eqv. set chi_L>chi_G entries to 0)

plt.plot(CHI_MID,W_L_list[:,100])
plt.xlabel('Chi [Mpc]')
plt.title('Lens kernel W_L with source at chi=%1.1f Mpc'%(CHI_MID[100]))
plt.tight_layout()
plt.savefig(os.path.join(outputpath,'lensing_kernels/W_L.png'))
plt.close()

plt.figure(figsize=(6,6))
W_i = np.empty((z_bin_num,len(CHI_MID)))
for i in range(z_bin_num):
    n_bin = n_true_chi_data['BIN%i'%(i+1)]
    for chi_L_index in range(len(CHI_MID)):
        #flat geometry (Troxel Ishak 3pt SC, below eqn6)
        W_i[i,chi_L_index] = np.sum(W_L_list[chi_L_index]*n_bin)
    plt.plot(CHI_MID,W_i[i],label='W_%i'%(i+1))
plt.xlabel('Chi [Mpc]') 
plt.title('Integrated kernels (W_i), with Chi in [Mpc]')
plt.legend(bbox_to_anchor=(1, 1.05))
plt.tight_layout()
plt.savefig(os.path.join(outputpath,'lensing_kernels/W_i.png'))
plt.close()



print('    - Interpolating P(k,z) for further calculations...')
matter_pow_p_k_interpolated = interpolate_2d(matter_pow_p_k,matter_pow_z,matter_pow_k_h,Z_MID,matter_pow_k_h)




Starting preliminary calculations
   - Converting distances data into functions
   - chi interval used is 14.074
   - Converting galaxy distribution in z-space to chi-space
   0
0  0
1  1
2  2
3  3
4  4
5  5
6  6
7  7
8  8
9  9
    - Interpolating P(k,z) for further calculations...


In [35]:
n_photo_chi_data.dtype
print()
test = np.empty((len(n_photo_chi_data),23))
for i in range(len(n_photo_chi_data)):
#     print(n_photo_chi_data[i])
    test[i] = list(n_photo_chi_data[i])
test = np.array(test,dtype = n_photo_chi_data.dtype)
print(test.shape)


(600, 23)


In [5]:
n_photo_chi_data.

ValueError: Big-endian buffer not supported on little-endian compiler

ValueError: Big-endian buffer not supported on little-endian compiler

In [ ]:
########################################
#Run CosmoSIS
########################################

########################################
#Load CosmoSIS data on matter pows etc.
########################################

#get cmv distance and lensing kernel (unit = Mpc)
#get integrated lensing kernel W_i(z_L) for each redsfhit bin (Zhang 2010 A2)
#chi tested, exactly what cosmoSIS produces
print(' - Computing cmv dists [Mpc], W_L, and integrated lensing kernels W_i')
[chi_list,W_L_list] = get_basic_cosmo_lensing(d_horiz,Omega0,OmegaLambda,Omegak,z_list,outputpath,USECOSMOSIS=True,data_path=os.path.join(outputpath, 'cosmoSIS_output/distances')); #in Mpc unit
W_i = get_integrated_lensing_kernel(W_L_list,true_z_file,z_list,z_bin_num,outputpath)

########################################
#Load CosmoSIS Output
########################################


print(' - Assuming CosmoSIS has been ran. Loading CosmoSIS output.')

#load hubble parameter
print('   - Loading h(z) from CosmoSIS, computing H(Z) [km/s/Mpc] and interpolating.')
#CosmoSIS only explicitly gives the little h, in unit Mpc^-1
#H = c 100 h km s^-1 Mpc^-1
distance_folder = os.path.join(outputpath,'cosmoSIS_output/distances')
with open(os.path.join(distance_folder,'z.txt'),'rb') as f:
    temp_z_h = np.loadtxt(f)
with open(os.path.join(distance_folder,'h.txt'),'rb') as f:
    temp_little_h = np.loadtxt(f)
hubble_func = interpolate.interp1d(temp_z_h,temp_little_h,kind='linear') 
hubble_list = hubble_func(z_list)*light_speed #this is big H(z) in unit of km/s/Mpc

#load the C_GG in angular space from CosmoSIS (shear-shear option)
print('   - Loading C_GG(ell) from CosmoSIS')
shear_cl_folder = os.path.join(outputpath,'cosmoSIS_output/shear_cl')
with open(os.path.join(shear_cl_folder,'ell.txt'),'rb') as f:
    ell_list = np.loadtxt(f)
C_GG_ii = np.empty((z_bin_num,len(ell_list)))
plt.subplots(figsize=(6,6))
for i in range(z_bin_num):
    with open(os.path.join(shear_cl_folder,'bin_%i_%i.txt'%(i+1,i+1)),'rb') as f:
        C_GG_ii[i] = np.loadtxt(f)
    plt.plot(ell_list,C_GG_ii[i],label='C_GG_ii, i=%i'%(i+1))
plt.xlabel('ell')
plt.ylabel('C_GG_ii')
plt.xscale('log')
plt.yscale('log')
plt.legend(bbox_to_anchor=(1, 0),loc='lower left') #bbox(x,y)
plt.tight_layout()
plt.savefig(os.path.join(outputpath,'combined_C_GG_ii.png'))
#plt.show()
plt.close()

#load the C_Gg in angular space from CosmoSIS (position-shear option)
print('   - Loading C_Gg(ell) from CosmoSIS')
galaxy_shear_cl_folder = os.path.join(outputpath,'cosmoSIS_output/galaxy_shear_cl')
with open(os.path.join(galaxy_shear_cl_folder,'ell.txt'),'rb') as f:
    ell_list = np.loadtxt(f)
C_Gg_ii = np.empty((z_bin_num,len(ell_list)))
plt.subplots(figsize=(6,6))
for i in range(z_bin_num):
    with open(os.path.join(galaxy_shear_cl_folder,'bin_%i_%i.txt'%(i+1,i+1)),'rb') as f:
        C_Gg_ii[i] = np.loadtxt(f)
    plt.plot(ell_list,C_Gg_ii[i],label='C_Gg_ii, i=%i'%(i+1))
plt.xlabel('ell')
plt.ylabel('C_Gg_ii')
plt.xscale('log')
plt.yscale('log')
plt.legend(bbox_to_anchor=(1, 0),loc='lower left') #bbox(x,y)
plt.tight_layout()
plt.savefig(os.path.join(outputpath,'combined_C_Gg_ii.png'))
#plt.show()
plt.close()

#load the C_gg in angular space from CosmoSIS (position-position option)
print('   - Loading C_gg(ell) from CosmoSIS')
galaxy_cl_folder = os.path.join(outputpath,'cosmoSIS_output/galaxy_cl')
with open(os.path.join(galaxy_cl_folder,'ell.txt'),'rb') as f:
    ell_list = np.loadtxt(f)
C_gg_ii = np.empty((z_bin_num,len(ell_list)))
plt.subplots(figsize=(6,6))
for i in range(z_bin_num):
    with open(os.path.join(galaxy_cl_folder,'bin_%i_%i.txt'%(i+1,i+1)),'rb') as f:
        C_gg_ii[i] = np.loadtxt(f)
    plt.plot(ell_list,C_gg_ii[i],label='C_gg_ii, i=%i'%(i+1))
plt.xlabel('ell')
plt.ylabel('C_gg_ii')
plt.xscale('log')
plt.yscale('log')
plt.legend(bbox_to_anchor=(1, 0),loc='lower left') #bbox(x,y)
plt.tight_layout()
plt.savefig(os.path.join(outputpath,'combined_C_gg_ii.png'))
#plt.show()
plt.close()

#get the power spectrum P(k,z) from CosmoSIS. (need non-linear spectrum)
#CosmoSIS unit: k is in (Mpc/h), P is in (h/Mpc)^3
#interpolating the P(k,z) to fit a z_list of Z_MID values
#matter_pow_p_k[z_index,kindex]
print('   - Loading P(k,z) from CosmoSIS.')
matter_power_folder = os.path.join(outputpath,'cosmoSIS_output/matter_power_nl')
with open(os.path.join(matter_power_folder, 'p_k.txt'),'rb') as f:
    matter_pow_p_k = np.loadtxt(f,delimiter = ' ') 
with open(os.path.join(matter_power_folder, 'k_h.txt'),'rb') as f:
    matter_pow_k_h = np.loadtxt(f,delimiter = ' ')
with open(os.path.join(matter_power_folder, 'z.txt'),'rb') as f:
    matter_pow_z = np.loadtxt(f,delimiter = ' ')
print(' - Interpolating P(k,z) for further calculations...')
matter_pow_p_k_interpolated = interpolate_2d(matter_pow_p_k,matter_pow_z,matter_pow_k_h,z_list,matter_pow_k_h)

In [ ]:
#